## Clustering draft 1

**General idea** : test the functions defined in the module1.py file and make a first application.

**General plan** : 
1. Import the data using the module YahooFinance 

2. Repeat the clustering several times to get several clustering outcomes

3. 

Here are the main package we use in this notebook

In [21]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from bs4 import BeautifulSoup
import requests 


We also import the module1.py file to use the function we defined there.

In [22]:
## we make some manipulations to correctly import module1

import os
import sys

# on a obtenu le chemin absolu vers notre répertoire en utilisant
# le terminal (ls, pwd, cd)
module_1_directory = '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/code'

# Ajouter le chemin du répertoire 'code' au chemin de recherche Python
sys.path.append(module_1_directory)

# Maintenant, vous pouvez importer module1
import module1

### 1. Importation of data

We download various types of data (Open, High, Low, Close, Adj Close, Volume) for 198 assets between two periods (start, end) into a pandas dataframe denoted as df.

We then store the returns of each of these assets in the 'data'.

#### 1.1. Scraping the names of the assets of interest

This is important because to import financial data for assets on Yahoo Finance, you need the tickers' names. This scraping work was performed in the file get_S&P500_symbols.ipynb and generated the S&P500_symbols.csv file.

In [23]:
import csv 

with open('S&P500_symbols.csv', mode='r') as file:
    # Create a CSV reader
    csv_reader = csv.reader(file)
    
    # Transform the CSV data into a list
    stock_symbols = []
    for row in csv_reader:
        stock_symbols.append(row[0])

# Close the file
file.close()

stock_symbols.pop(0)

'Symbols of S&P 500'

In [24]:
stock_symbols 

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADM',
 'ADBE',
 'ADP',
 'AES',
 'AFL',
 'A',
 'ABNB',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BG',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',


#### 1.2. Creating the dataframe using pandas and yfinance 

In this case, we focus on the financial data of asset during the year 2022

In [25]:
n_stocks = len(stock_symbols) # number of stocks = 502

start = "2022-01-01" # start date
end = "2022-12-01" # end date

df = pd.DataFrame(yf.download(stock_symbols, start, end)) # data on the 198 assets
data = np.log(df["Close"]/df["Open"]).transpose() # compute the returns of these assets
data = data.dropna()

[*********************100%%**********************]  502 of 502 completed


5 Failed downloads:


['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-01-01 -> 2022-12-01)')
['GEHC', 'KVUE', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1669870800")
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


Due to missing values in the data, 5 stocks have been excluded from the intial 502 we considered. Thus the shape of the dataframe is (496, 230) 

#### 1.2. We repeat the clustering step several times to start preparing multiple micro-portfolios

**General idea**: The multiple_clustering function takes as arguments:

1. *n_repeat*: an integer that corresponds to the number of times we want to train (.fit()) the model passed as an argument on the data.

2. *data*: a pandas dataframe (like the one we generated above) that corresponds to the financial data on which the model will be trained.

3. *model*: a sklearn clustering model (for now, we will test it with KMeans).

4. *model_name*: the name of the model, necessary for creating the pipeline properly.

**Outcome**: a pandas dataframe storing the results of the multiple clustering and a dictionary containing the centroids of the clusters each time 

We first make the test using the K-Means model and fix we fix the number of clusters to 5

In [26]:
model = KMeans(n_clusters=5)
model_name = 'kmeans'

Y, C = module1.multiple_clusterings(10, data, model, model_name)
print(Y, C)

/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

      Clustering n°1  Clustering n°2  Clustering n°3  Clustering n°4  \
A                  0               1               4               4   
AAL                3               0               2               3   
AAPL               0               1               4               4   
ABBV               1               3               0               2   
ABNB               3               0               2               3   
...              ...             ...             ...             ...   
YUM                0               1               4               2   
ZBH                0               1               4               4   
ZBRA               0               4               1               1   
ZION               0               1               4               4   
ZTS                0               1               4               4   

      Clustering n°5  Clustering n°6  Clustering n°7  Clustering n°8  \
A                  4               1               2           

/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


#### 1.3. Get the cluster compositions each time 

**Why doing so?**: the purpose of this work is to compute the returns of each cluster (seen as a new "fictive" asset composed of "real" assets) and this latter task is easier if we know the cluster composition in terms of tickers (because in the financial database we have data given a ticker).

In [27]:
Y_symbol = module1. cluster_composition(Y)
Y_symbol

,Clustering n°1,Clustering n°2,Clustering n°3,Clustering n°4,Clustering n°5,Clustering n°6,Clustering n°7,Clustering n°8,Clustering n°9,Clustering n°10
Cluster 1,"[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, AES, AIG, AKAM, ALLE, AME,...","[A, AAPL, ABT, ACN, ADI, ADP, AES, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, AAPL, ABT, ACN, ADP, AES, AIG, AKAM, ALLE,...","[A, ACN, AES, AIG, AKAM, ALLE, AMCR, AME, AMP,...","[A, AAPL, ABT, ACN, ADP, AES, AJG, AKAM, ALLE,...",NaN
Cluster 2,"[AAL, ABNB, ALK, BA, BBWI, BKNG, CCL, CZR, DAL...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, AIG, ALK, AMP, APTV, AXP, BA, BAC, BBWI,...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ALK, APTV, BA, BBWI, BKNG, CCL, CZ...","[AAL, ABNB, ADBE, ADI, ADSK, ALB, ALGN, AMAT, ...",NaN
Cluster 3,"[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, ADP, AEE, AEP, AFL, AIZ, AJG...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, AJG, ALL...","[AAPL, ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, ...","[ABBV, ACGL, ADM, AEE, AEP, AFL, AIZ, ALL, AMC...",NaN
Cluster 4,"[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ABNB, ADBE, ADSK, ALB, ALGN, AMAT, AMD, AMZN,...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ADBE, ADI, ADSK, ALB, ALGN, AMAT, AMD, AMZN, ...","[ABBV, ABT, ACGL, ADM, ADP, AEE, AEP, AFL, AIZ...","[AIG, ALK, AMP, AVY, AXP, BA, BAC, BBY, BEN, B...",NaN
Cluster 5,"[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...","[APA, BKR, CF, COP, CTRA, CVX, DVN, EOG, EQT, ...",NaN


#### 1.4. Compute the weights of each assets within a cluster

**Why doing so?**: because each stock in a cluster has a different weight. Stock are "representative" of the cluster in which they live to a different extent. So far, we built the weight of a stock based on its $L^2$ distance from the cluster's centroid. We can therefore see the weights as a kind of "inertia contribution" of the stock in the cluster.

In [30]:
cluster = Y_symbol.iloc[0,0]

centroid = C.iloc[0,0]

weights = module1.cluster_weights(cluster, centroid, data)

In [32]:
weights

,0,1,2,3,4,5,6,7,8,9,...,234,235,236,237,238,239,240,241,242,243
0,0.507112,0.499517,0.502985,0.504042,0.495568,0.488757,0.501108,0.498078,0.511769,0.505492,...,0.504534,0.501387,0.509895,0.502691,0.51005,0.501059,0.501915,0.514851,0.517077,0.501996


In [33]:
def cluster_return(cluster, weights):
    '''
    ----------------------------------------------------------------
    GENERAL IDEA : each cluster is seen as a new asset and the goal 
                   of this routine is to compute the return of this 
                   asset (cluster) given its compositions and weights 
                   put on the different sub-assets that compose this 
                   cluster
    ----------------------------------------------------------------

    ----------------------------------------------------------------
    PARAMS : 
    
    - cluster : pandas dataframe composed of tickers (strings) corresponding to the stocks 
                that compose this cluster 
                [shape : (1, n_stocks_in_cluster)]
                
    
    - weights : pandas dataframe composed of floats that correspond to the weights of each 
                tickers in the cluster
                !! Note that the i-th component of the weights list 
                corresponds to the weight of the i-th ticker in the 
                list cluster !! 
                [shape : (1, n_stocks_in_cluster)]

                
    - return_data : pandas dataframe containing the return of the 
                    stocks 
                    [shape : (n_stocks_in_cluster, n_days_observed)]
    ----------------------------------------------------------------
    '''
    tickers_data = data.loc[cluster]
    result_data = pd.DataFrame(weights.values.dot(tickers_data.values), columns=tickers_data.columns)

    return result_data
    

def cluster_portfolio_return(cluster_composition, weights_matrix, return_data):
    '''
    ----------------------------------------------------------------
    GENERAL IDEA : compute the return of a portfolio composed of 
                   assets (which are clusters) by using the 
                   cluster_return function.
    ----------------------------------------------------------------

    ----------------------------------------------------------------
    PARAMS : 

    - cluster_composition : pandas dataframe. Corresponds to ONE ROW
                            of the dataframe returned by the 
                            cluster_composition function
                            [shape : (n_clusters, 1)]
                
    
    - weights_matrix : pandas dataframe, the weights are distributed in 
                       the same order as the stock_symbols list
                       [shape (1, n_stocks_in_cluster)]

                
    - return_data : pandas dataframe containing the return of the 
                    stocks 
                    [shape (n_stocks_in_cluster, n_days_observed)]
    ----------------------------------------------------------------
    '''
    
    n_clusters = cluster_composition.shape 
    
    micro_portfolio_return = pd.DataFrame(index=cluster_composition.index, columns=return_data.columns).transpose()
    
    for i in range(n_clusters):
        cluster = cluster_composition[cluster_composition.index[i]] ## get all the tickers in one cluster
        
        coordonnee_tickers = [stock_symbols.index(element) for element in cluster]

        weight_cluster = pd.DataFrame(weights_matrix[coordonnee_tickers])

        micro_portfolio_return[cluster_composition.index[i]] = cluster_return(cluster, weight_cluster).transpose()
        
    return micro_portfolio_return.transpose()
            

In [34]:
print(cluster_return(cluster, weights))

Date  2022-01-03  2022-01-04  2022-01-05  2022-01-06  2022-01-07  2022-01-10  \
0      -0.605718    0.877205   -2.239903   -0.004645   -0.636896   -0.314597   

Date  2022-01-11  2022-01-12  2022-01-13  2022-01-14  ...  2022-11-16  \
0       0.969862   -0.291764   -0.908531    0.244197  ...   -0.897202   

Date  2022-11-17  2022-11-18  2022-11-21  2022-11-22  2022-11-23  2022-11-25  \
0        0.92125   -0.698403    0.394256    0.982179    0.607733    0.335126   

Date  2022-11-28  2022-11-29  2022-11-30  
0      -1.404862    0.392251    3.036058  

[1 rows x 230 columns]


In [35]:
index_names = ['Cluster %i'%(i+1) for i in range(n_clusters)]

NameError: name 'n_clusters' is not defined

In [36]:
C = B.loc[:, 'Clustering n°1'] ## format que l'on entre dans la fonction 
list_1 = B.iloc[0, 0] ## un cluster 

NameError: name 'B' is not defined

In [59]:
index_names = ['Cluster %i'%(i+1) for i in range(5)]
index_names

['Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5']

In [96]:
cluster = C[B.index[0]]

## beware of the shape of the weights vector and its type (has to be a pandas dataframe)

weights = pd.DataFrame(np.arange(1, 34)/sum(np.arange(1, 34))).transpose()

## we create a pandas dataframe to store the results but 
## we first transpose it to add the data more easily
micro_portfolio_return = pd.DataFrame(index=B.index, columns=data.columns).transpose()

D = cluster_return(cluster, weights)


In [97]:
micro_portfolio_return.transpose()

Date,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-10,2022-01-11,2022-01-12,2022-01-13,2022-01-14,...,2022-11-16,2022-11-17,2022-11-18,2022-11-21,2022-11-22,2022-11-23,2022-11-25,2022-11-28,2022-11-29,2022-11-30
Cluster 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cluster 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cluster 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cluster 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cluster 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
micro_portfolio_return[B.index[0]] = D.transpose()

In [ ]:
df = df.append(, ignore_index=True)

In [27]:
data.loc[list_1]

Date,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-10,2022-01-11,2022-01-12,2022-01-13,2022-01-14,...,2022-11-16,2022-11-17,2022-11-18,2022-11-21,2022-11-22,2022-11-23,2022-11-25,2022-11-28,2022-11-29,2022-11-30
AAPL,0.023234,-0.016173,-0.026459,-0.004061,-0.004173,0.018227,0.015890,-0.003356,-0.020635,0.010046,...,-0.002283,0.028876,-0.006719,-0.014422,0.013744,0.010781,-0.001349,-0.006359,-0.021860,0.045822
GOOGL,-0.000438,-0.006877,-0.047104,0.005452,-0.008202,0.026241,0.012450,0.001985,-0.021084,0.017367,...,0.009657,0.014233,-0.013660,-0.017523,0.011399,0.014012,-0.007971,-0.010254,-0.005657,0.063041
TSLA,0.044335,-0.034170,-0.052393,-0.011486,-0.050701,0.056494,0.010132,0.025053,-0.072450,0.028734,...,-0.024259,-0.004304,-0.026614,-0.046442,0.007562,0.053997,-0.011959,0.016314,-0.022744,0.065093
JPM,0.011444,0.021197,-0.024368,-0.008363,0.008954,-0.010405,-0.004587,-0.000119,-0.000654,-0.018014,...,0.000902,0.004083,-0.005440,-0.005173,0.007731,0.011348,0.001903,-0.012721,0.014011,0.015241
AVGO,-0.004513,0.012056,-0.042937,-0.004901,-0.026615,0.020000,0.010521,-0.008820,-0.049460,0.000990,...,-0.009704,0.013981,-0.003314,0.008735,0.013447,0.006184,-0.001471,-0.005785,-0.003083,0.050088
LLY,-0.009814,-0.008175,-0.028405,-0.002965,-0.000924,0.007409,0.002787,0.011871,-0.017470,-0.024401,...,0.008337,0.022028,-0.002430,0.002162,-0.003099,0.001051,0.006455,0.006363,-0.001839,0.016329
CVX,0.015549,0.012263,-0.000164,-0.002916,0.011908,-0.000320,0.016626,-0.005170,-0.002678,0.017207,...,-0.008344,0.014114,0.011155,0.008366,0.011579,0.005606,-0.006457,-0.010707,0.004263,0.002513
MRK,0.003780,0.007953,0.020751,0.000508,0.018096,0.022467,-0.009141,0.001479,0.004807,-0.002700,...,0.003509,0.018746,0.015470,0.005793,0.007229,0.000656,0.008783,0.012246,0.007748,0.012979
KO,0.008127,0.009332,0.011582,0.000000,0.000829,-0.001984,0.000331,0.000165,0.005929,0.007357,...,-0.004451,0.008602,0.002456,0.010851,-0.001923,0.002238,-0.000638,0.000160,-0.000640,0.025475
DHR,-0.035345,-0.018509,-0.013663,-0.002071,-0.023862,0.029572,0.000688,0.000000,-0.042731,0.011763,...,-0.014324,-0.014037,-0.016815,0.000923,-0.002287,-0.000152,0.007820,-0.014861,-0.003776,0.054144
